In [1]:
#importing libraries
from keras.preprocessing import text
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [2]:
#taking random sentences as data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. 
Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
"""
dl_data = data.split()

In [3]:
#tokenization
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(dl_data)
word2id = tokenizer.word_index

word2id['PAD'] = 0
id2word = {v:k for k, v in word2id.items()}
wids = tokenizer.texts_to_sequences(dl_data)

vocab_size = len(word2id)
embed_size = 100
window_size = 2 

print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

Vocabulary Size: 75
Vocabulary Sample: [('learning', 1), ('deep', 2), ('networks', 3), ('neural', 4), ('and', 5), ('as', 6), ('of', 7), ('machine', 8), ('supervised', 9), ('have', 10)]


In [4]:
#generating (context word, target/label word) pairs
def cbow_model(corpus, window_size, vocab_size):
    context_length = window_size*2
    for words in corpus:
        sentence_length = len(words)
        for index, word in enumerate(words):
            context_words = []
            label_word   = []            
            start = index - window_size
            end = index + window_size + 1
            
            context_words.append([words[i] 
                                 for i in range(start, end) 
                                 if 0 <= i < sentence_length 
                                 and i != index])
            label_word.append(word)

            x = pad_sequences(context_words, maxlen=context_length)
            y = np_utils.to_categorical(label_word, vocab_size)
            yield (x, y)

In [5]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

cbow = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embed_size, input_length=window_size*2),
    Lambda(lambda x: K.mean(x, axis=1), output_shape=(embed_size,)),
    Dense(vocab_size, activation='softmax'),
])

cbow.compile(loss='categorical_crossentropy', optimizer='adam')

cbow.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            7500      
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 75)                7575      
                                                                 
Total params: 15,075
Trainable params: 15,075
Non-trainable params: 0
_________________________________________________________________


In [6]:
for epoch in range(1, 6):
    loss = 0.
    i = 0
    for x, y in cbow_model(corpus=wids, window_size=window_size, vocab_size=vocab_size):
        i += 1
        loss += cbow.train_on_batch(x, y)
        if i % 100000 == 0:
            print('Processed {} (context, word) pairs'.format(i))

    print('Epoch:{}  Loss - > {}\n'.format(epoch,loss))

Epoch:1  Loss - > 435.42456579208374

Epoch:2  Loss - > 428.598828792572

Epoch:3  Loss - > 425.9633219242096

Epoch:4  Loss - > 423.121390581131

Epoch:5  Loss - > 420.4714775085449



In [7]:
weights = cbow.get_weights()[0]
weights = weights[1:]
# print(weights.shape)

pd.DataFrame(weights, index=list(id2word.values())[1:]).head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
deep,0.025099,0.002957,-0.066654,-0.074473,0.020489,-0.057802,-0.052813,-0.048676,0.051715,-0.100841,...,-0.064430,-0.044600,-0.019513,-0.007621,0.052932,-0.018841,0.056269,0.065961,0.063113,-0.050300
networks,0.068969,0.020312,-0.083768,-0.087222,-0.002124,0.053218,0.066224,0.018786,0.018349,-0.024450,...,-0.006085,-0.029850,0.040946,0.046921,0.034693,-0.063099,-0.053378,0.086499,0.015266,0.000254
neural,0.026651,0.014363,0.039218,0.044278,-0.002899,0.016150,0.049927,0.039386,0.048270,-0.015757,...,0.036546,0.020022,-0.045873,0.039297,-0.007303,0.024353,0.030147,-0.026098,-0.026410,-0.037111
and,0.011125,-0.006770,0.042212,0.034024,-0.045635,-0.025348,-0.034013,-0.022987,-0.001175,-0.037266,...,-0.014275,-0.008384,-0.008142,-0.026061,-0.045640,-0.003847,0.028368,-0.042055,-0.011914,-0.019121
as,0.034273,0.037756,-0.023268,0.036076,0.011957,0.040964,-0.039998,-0.009107,0.026859,-0.013804,...,0.049830,-0.029716,-0.049701,0.020268,0.003873,-0.046448,-0.029851,0.007500,0.040979,0.046728


In [8]:
from sklearn.metrics.pairwise import euclidean_distances

distance_matrix = euclidean_distances(weights)
print(distance_matrix.shape)

similar_words = {search_term: [id2word[idx] for idx in 
                               distance_matrix[word2id[search_term]-1].argsort()[1:6]+1] 
                   for search_term in ['as','in','to']}

similar_words

(74, 74)


{'as': ['results', 'performance', 'convolutional', 'structured', 'language'],
 'in': ['broader', 'be', 'transformers', 'climate', 'natural'],
 'to': ['based', 'science', 'representation', 'with', 'comparable']}